# 飞桨常规赛：PALM眼底彩照中黄斑中央凹定位 - 9月第2名方案

# 赛题介绍
PALM黄斑定位常规赛的重点是研究和发展与患者眼底照片黄斑结构定位相关的算法。该常规赛的目标是评估和比较在一个常见的视网膜眼底图像数据集上定位黄斑的自动算法。具体目的是预测黄斑中央凹在图像中的坐标值。
图

# 数据简介
PALM病理性近视预测常规赛由中山大学中山眼科中心提供800张带黄斑中央凹坐标标注的眼底彩照供选手训练模型，另提供400张带标注数据供平台进行模型测试。

# 数据说明
本次常规赛提供的金标准由中山大学中山眼科中心的7名眼科医生手工进行标注，之后由另一位高级专家将它们融合为最终的标注结果。本比赛提供数据集对应的黄斑中央凹坐标信息存储在xlsx文件中，名为“Fovea_Location_train”，第一列对应眼底图像的文件名(包括扩展名“.jpg”)，第二列包含x坐标，第三列包含y坐标。
图

# 训练数据集
文件名称：Train
Train文件夹里有一个文件夹fundus_images和一个xlsx文件。

fundus_images文件夹内包含800张眼底彩照，分辨率为1444×1444，或2124×2056。命名形如H0001.jpg、P0001.jpg、N0001.jpg和V0001.jpg。
xlsx文件中包含800张眼底彩照对应的x、y坐标信息。
测试数据集
文件名称：PALM-Testing400-Images 文件夹里包含400张眼底彩照，命名形如T0001.jpg

# 赛题重点难点
这个项目解决的是一个回归问题，label是中央凹的坐标，通过卷积神经网络提取特征来提取中央凹坐标。难点是如果通过数据预处理，模型选择，loss选择，优化器选取，学习率调整来提高特征提取的准确度

# 整体思路

# 把常规赛：PALM眼底彩照中黄斑中央凹定位数据集解压到work文件夹下

# 1、数据预处理
去掉训练集和验证集里面的label是（0,0）的样本，resize成（224,224）,label是（0,0）如果直接作为训练样本，因为（0,0）并不是真实的坐标，直接加入训练会降低训练结果的准确性。
通过改变亮度，扩展，裁剪，随即翻转，随机插值等进行数据增强
```
def random_distort(img):
    
    # 随机改变亮度
    def random_brightness(img, lower=0.5, upper=1.5):
        e = np.random.uniform(lower, upper)
        return ImageEnhance.Brightness(img).enhance(e)
    # 随机改变对比度
    def random_contrast(img, lower=0.5, upper=1.5):
        e = np.random.uniform(lower, upper)
        return ImageEnhance.Contrast(img).enhance(e)
    # 随机改变颜色
    def random_color(img, lower=0.5, upper=1.5):
        e = np.random.uniform(lower, upper)
        return ImageEnhance.Color(img).enhance(e)

    ops = [random_brightness, random_contrast, random_color]
    np.random.shuffle(ops)

    img = Image.fromarray(img)
    img = ops[0](img)
    img = ops[1](img)
    img = ops[2](img)
    img = np.asarray(img)

    return img



def random_expand(img,
                  label,
                  max_ratio=3.,
                  fill=None,
                  keep_ratio=True,
                  thresh=0.5):
    if random.random() > thresh:
        return img, label

    if max_ratio < 1.0:
        return img, label

    h, w, c = img.shape
    ratio_x = random.uniform(1, max_ratio)
    if keep_ratio:
        ratio_y = ratio_x
    else:
        ratio_y = random.uniform(1, max_ratio)
    oh = int(h * ratio_y)
    ow = int(w * ratio_x)
    off_x = random.randint(0, ow - w)
    off_y = random.randint(0, oh - h)

    out_img = np.zeros((oh, ow, c))
    if fill and len(fill) == c:
        for i in range(c):
            out_img[:, :, i] = fill[i] * 255.0

    out_img[off_y:off_y + h, off_x:off_x + w, :] = img
    label[0] = ((label[0] * w) + off_x) / float(ow)
    label[1] = ((label[1] * h) + off_y) / float(oh)


    return out_img.astype('uint8'), label


def random_crop(img,label):
    if random.random() > 0.5:
        return img, label
    img1=img.copy()
    label1=label.copy()
    h,w,c=img1.shape
    ow=int(0.7*w)
    oh=int(0.7*h)
    offx=random.randint(0, w-ow)
    offy=random.randint(0, h-oh)
    img2=img1[offy:offy+oh,offx:offx+ow]

    label1[0] = ((label1[0] * w)-offx) / ow
    label1[1] = ((label1[1] * h)-offy) / oh

    if label1[0]<0 or label1[0]>1 or label1[1]<0 or label1[1]>1:
        return img, label


    return img2,label1



def random_interp(img, size):
    interp_method = [
        cv2.INTER_NEAREST,
        cv2.INTER_LINEAR,
        cv2.INTER_AREA,
        cv2.INTER_CUBIC,
        cv2.INTER_LANCZOS4,
    ]
    n=random.randint(0,4)
   
    img = cv2.resize(img,(size,size),interpolation = interp_method[2])
    return img



# 随机翻转
def random_flip(img, label, thresh=0.5):
    

    if random.random() > thresh:
        img = img[:, ::-1, :]
        if label!=[0,0]:
            label[0] = 1.0 - label[0]
    if random.random() > thresh:
        img = img[::-1, :, :]
        if label!=[0,0]:
            label[1] = 1.0 - label[1]
    
    return img, label



def image_augment(img, label, size, means=None):
    # 随机改变亮暗、对比度和颜色等
    img = random_distort(img)
    # 随机填充
    #img, label= random_expand(img, label, fill=means)
    # 随机裁剪
    img, label = random_crop(img, label)
    # 随机缩放
    img = random_interp(img, size)
    # 随机翻转
    img, label = random_flip(img, label)
   

    return img, label
```

# 2、模型主体搭建
采用resnet101主体网络
```
class resnet_model(paddle.nn.Layer):
    def __init__(self,num):
        super(resnet_model,self).__init__()
        self.model=resnet101(pretrained=True)
        self.fc=Linear(1000,num)
       

    def forward(self,x):
        out=self.model(x)
        out=self.fc(out)
        out=F.sigmoid(out)
        return out
```



# 3、loss选择

```
labelx=label[:,0]*w
labely=label[:,1]*h
            
outx=out[:,0]*w
outy=out[:,1]*h
           
square_x=F.square_error_cost(labelx,outx)
square_y=F.square_error_cost(labely,outy)
           
sqrt_xy=paddle.sqrt(square_x+square_y)
             
distance=paddle.mean(sqrt_xy)
 ```



# 4、优化器选择
```
lr = paddle.optimizer.lr.CosineAnnealingDecay(learning_rate=0.0001,T_max=int(640/batch_num) * epoches,verbose=False)
opt = paddle.optimizer.Adam(learning_rate=lr, parameters=model.parameters(),weight_decay=paddle.regularizer.L2Decay(0.0001))
```

# 主体网络

In [1]:
import paddle
import os
import cv2
import numpy as np
from paddle.vision.models import resnet152,resnet50,resnet101
from paddle.nn import Linear
import paddle.nn.functional as F
from paddle.optimizer import Momentum
from paddle.regularizer import L2Decay
from paddle.nn import CrossEntropyLoss
from paddle.metric import Accuracy, Auc
import time
import matplotlib.pyplot as plt
from paddle.vision.transforms import RandomHorizontalFlip,RandomVerticalFlip
from PIL import ImageEnhance,Image
import random
import matplotlib.image as imgplt
import blackhole.dataframe as df
import warnings
import pdb

warnings.simplefilter('ignore')

# 随机改变亮暗、对比度和颜色等
def random_distort(img):
    
    # 随机改变亮度
    def random_brightness(img, lower=0.5, upper=1.5):
        e = np.random.uniform(lower, upper)
        return ImageEnhance.Brightness(img).enhance(e)
    # 随机改变对比度
    def random_contrast(img, lower=0.5, upper=1.5):
        e = np.random.uniform(lower, upper)
        return ImageEnhance.Contrast(img).enhance(e)
    # 随机改变颜色
    def random_color(img, lower=0.5, upper=1.5):
        e = np.random.uniform(lower, upper)
        return ImageEnhance.Color(img).enhance(e)

    ops = [random_brightness, random_contrast, random_color]
    np.random.shuffle(ops)

    img = Image.fromarray(img)
    img = ops[0](img)
    img = ops[1](img)
    img = ops[2](img)
    img = np.asarray(img)

    return img



def random_expand(img,
                  label,
                  max_ratio=3.,
                  fill=None,
                  keep_ratio=True,
                  thresh=0.5):
    if random.random() > thresh:
        return img, label

    if max_ratio < 1.0:
        return img, label

    h, w, c = img.shape
    ratio_x = random.uniform(1, max_ratio)
    if keep_ratio:
        ratio_y = ratio_x
    else:
        ratio_y = random.uniform(1, max_ratio)
    oh = int(h * ratio_y)
    ow = int(w * ratio_x)
    off_x = random.randint(0, ow - w)
    off_y = random.randint(0, oh - h)

    out_img = np.zeros((oh, ow, c))
    if fill and len(fill) == c:
        for i in range(c):
            out_img[:, :, i] = fill[i] * 255.0

    out_img[off_y:off_y + h, off_x:off_x + w, :] = img
    label[0] = ((label[0] * w) + off_x) / float(ow)
    label[1] = ((label[1] * h) + off_y) / float(oh)


    return out_img.astype('uint8'), label


def random_crop(img,label):
    if random.random() > 0.5:
        return img, label
    img1=img.copy()
    label1=label.copy()
    h,w,c=img1.shape
    ow=int(0.7*w)
    oh=int(0.7*h)
    offx=random.randint(0, w-ow)
    offy=random.randint(0, h-oh)
    img2=img1[offy:offy+oh,offx:offx+ow]

    label1[0] = ((label1[0] * w)-offx) / ow
    label1[1] = ((label1[1] * h)-offy) / oh

    if label1[0]<0 or label1[0]>1 or label1[1]<0 or label1[1]>1:
        return img, label


    return img2,label1



def random_interp(img, size):
    interp_method = [
        cv2.INTER_NEAREST,
        cv2.INTER_LINEAR,
        cv2.INTER_AREA,
        cv2.INTER_CUBIC,
        cv2.INTER_LANCZOS4,
    ]
    n=random.randint(0,4)
   
    img = cv2.resize(img,(size,size),interpolation = interp_method[2])
    return img



# 随机翻转
def random_flip(img, label, thresh=0.5):
    

    if random.random() > thresh:
        img = img[:, ::-1, :]
        if label!=[0,0]:
            label[0] = 1.0 - label[0]
    if random.random() > thresh:
        img = img[::-1, :, :]
        if label!=[0,0]:
            label[1] = 1.0 - label[1]
    
    return img, label



def image_augment(img, label, size, means=None):
    # 随机改变亮暗、对比度和颜色等
    img = random_distort(img)
    # 随机填充
    #img, label= random_expand(img, label, fill=means)
    # 随机裁剪
    img, label = random_crop(img, label)
    # 随机缩放
    img = random_interp(img, size)
    # 随机翻转
    img, label = random_flip(img, label)
   

    return img, label


data_dir = 'work/常规赛：PALM眼底彩照中黄斑中央凹定位/Train'
data_dir1='work/常规赛：PALM眼底彩照中黄斑中央凹定位'

def get_annotations():
    train_dir=os.path.join(data_dir,'train.xlsx')
    valid_dir=os.path.join(data_dir,'valid.xlsx')
    test_dir=os.path.join(data_dir1,'PALM-Testing400-Images')
    train_data = df.read_excel(train_dir)
    valid_data=df.read_excel(valid_dir)
    test_list=[]
  
    
    train_values=train_data.values
    valid_values=valid_data.values
   
    test=os.listdir(test_dir)
    test_list=[[a,-1,-1] for a in test]
    test_values=np.array(test_list,dtype=object)


    return train_values,valid_values,test_values


def get_image_data(value,mode='train'):
    imageName=value[0]
    label=value[1:3]
    if mode == 'train' or mode=='valid':
        imageDir=os.path.join(data_dir,'fundus_image',imageName)
    if mode =='test':
        imageDir=os.path.join(data_dir1,'PALM-Testing400-Images',imageName)
    imageData=imgplt.imread(imageDir)
    h,w,c=imageData.shape
    label_normal=[label[0]/w,label[1]/h]
    return imageData,label_normal,w,h


def normalize(img,label):
    mean=[0.485, 0.456, 0.406]
    #mean=[0.5,0.5,0.5]
    #std=[0.5,0.5,0.5]

    std=[0.229, 0.224, 0.225]
    mean=np.array(mean).reshape((1,1,-1))
    std=np.array(std).reshape((1,1,-1))
    label=np.array(label).astype('float32')
    img_Normal=(img/255.0-mean)/std
    img_Normal=np.transpose(img_Normal,(2,0,1)).astype('float32')

    return img_Normal,label


class dataset(paddle.io.Dataset):
    def __init__(self,mode='train',annotations=None):
        super(dataset,self).__init__()
        self.values=annotations
        self.mode=mode

    def __getitem__(self,idx):
        value=self.values[idx]
        img,Label,w,h=get_image_data(value,self.mode)
        if self.mode=='train':
            img,Label=image_augment(img, Label, 224, means=None)
        else:
            img = cv2.resize(img,(224,224), interpolation =cv2.INTER_AREA)
        image_normal,Label_normal=normalize(img,Label)
        return image_normal,Label_normal,w,h


    def __len__(self):
        return len(self.values)


class resnet_model(paddle.nn.Layer):
    def __init__(self,num):
        super(resnet_model,self).__init__()
        self.model=resnet101(pretrained=True)
        self.fc=Linear(1000,num)
       

    def forward(self,x):
        out=self.model(x)
        out=self.fc(out)
        out=F.sigmoid(out)
        return out


def train(model):
    batch_num=40
    epoches=200
    dataAnno=get_annotations()
    lr = paddle.optimizer.lr.CosineAnnealingDecay(learning_rate=0.0001,
                                                T_max=int(640/batch_num) * epoches,verbose=False)
    opt = paddle.optimizer.Adam(learning_rate=lr, parameters=model.parameters(),weight_decay=paddle.regularizer.L2Decay(0.0001))
    
    train_dataset=dataset(mode='train',annotations=dataAnno[0])
    
    train_loader=paddle.io.DataLoader(train_dataset, batch_size=batch_num, shuffle=True, num_workers=0, drop_last=False)
    valid_dataset=dataset(mode='valid',annotations=dataAnno[1])
    valid_loader=paddle.io.DataLoader(valid_dataset, batch_size=batch_num, shuffle=False, num_workers=0, drop_last=False)
    
    model.train()
    losses = []
    distances=[]
    for epoch in range(epoches):

        for batch_id,batch in enumerate(train_loader()):
            x=batch[0]
            w=batch[2]
            h=batch[3]
            label=batch[1]
            out=model(x)
            
            
            labelx=label[:,0]*w
            labely=label[:,1]*h
            
            outx=out[:,0]*w
            outy=out[:,1]*h
           
            square_x=F.square_error_cost(labelx,outx)
            square_y=F.square_error_cost(labely,outy)
           
            sqrt_xy=paddle.sqrt(square_x+square_y)
            
            distance=paddle.mean(sqrt_xy)

            loss=F.smooth_l1_loss(out,label)
            avg_loss=paddle.mean(loss)


            if batch_id % 2 == 0:
                print("train:epoch: {}, batch_id: {}, loss is: {:.5f}".format(epoch, batch_id, avg_loss.numpy()[0]))
            # 反向传播，更新权重，清除梯度
            distance.backward()
            opt.step()
            opt.clear_grad()
            lr.step()

        

        

        model.eval()
      
       
        losses1=[]
        distance1=[]
        for batch_id, data in enumerate(valid_loader()):
            img=data[0]
            w=data[2]
            h=data[3]
            label=data[1]
            
            out = model(img)
           
            labelx=label[:,0]*w
            labely=label[:,1]*h
            
            outx=out[:,0]*w
            outy=out[:,1]*h
           
            square_x=F.square_error_cost(labelx,outx)
            square_y=F.square_error_cost(labely,outy)
           
            sqrt_xy=paddle.sqrt(square_x+square_y)
            
            distance=paddle.mean(sqrt_xy)
           


            
            loss=F.smooth_l1_loss(out,label)
            avg_loss=paddle.mean(loss)
           
            
            #acc = paddle.metric.accuracy(pred, label1)
            losses1.append(avg_loss.numpy()[0])
            distance1.append(distance.numpy()[0])
            #accuracies.append(acc.numpy())
            
            print("[validation] loss: {:.5f} ,Diatance is {:.5f}".format(avg_loss.numpy()[0],distance.numpy()[0]))
        losses.append(sum(losses1)/len(losses1))
        distances.append(sum(distance1)/len(distance1))
        epo=np.argmin(np.array(losses))
        epo1=np.argmin(np.array(distances))
        if epo1==epoch:
            paddle.save(model.state_dict(), 'work/output/best_epoch'+str(epo1)+'.pdparams')
            paddle.save(opt.state_dict(), 'work/output/best_epoch'+str(epo1)+'.pdopt')

        print('验证集最小loss对应的epoch是：',epo,' loss值是：',losses[epo],' ; ','最小欧式距离对应的epoch是：',epo1,'distance是:',distances[epo1])
        lossss=open('work/loss.txt','a+')
        lossss.writelines('验证集最小loss对应的epoch是：'+str(epo)+' loss值是：'+str(losses[epo])+'  最小欧式距离对应的epoch是：'+str(epo1)+'  distance是:'+str(distances[epo1])+'\n')
        lossss.close()
        model.train()





def predict(model):
   
    dataAnno=get_annotations()
    model_pararams=paddle.load('work/checkpoint/179.pdparams')
    model.load_dict(model_pararams)
    model.eval()
    test_dataset=dataset(mode='test',annotations=dataAnno[2])
    batch=10
    test_loader=paddle.io.DataLoader(test_dataset, batch_size=batch, shuffle=False, num_workers=0, drop_last=False)
    Fovea_XY=[]
    sizeList=[]
    for batch_id,batch in enumerate(test_loader()):
        x=batch[0]      
        sizeList.append(np.array([batch[2].numpy(),batch[3].numpy()]).transpose())
        out=model(x)
        result=out.numpy()
        Fovea_XY.append(result)
        print('预测到：batch',batch_id)

    Fovea_XY=np.concatenate(Fovea_XY,axis=0)
    sizeList=np.concatenate(sizeList,axis=0)
    FileName=get_annotations()[2][:,0]
    Fovea_XY=Fovea_XY*sizeList
    Fovea_X=Fovea_XY[:,0]
    Fovea_Y=Fovea_XY[:,1]
   
    submission = df.DataFrame(data={
                            "FileName": FileName,
                            "Fovea_X": Fovea_X,
                            "Fovea_Y": Fovea_Y
                        })
    submission=submission.sort_values(by='FileName')
    submission.to_csv("work/Fovea_Localization_Results.csv", index=False)





paddle.device.set_device('GPU:0')

model=resnet_model(2)
#train(model)
predict(model)



/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/blackhole/dataframe/typedef/typehints.py:157: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dt.stype.int32: [int, 'int', np.int, np.int32],
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/blackhole/dataframe/typedef/typehints.py:159: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use


# 飞桨使用体验及给其他选手的一些建议
飞桨是一个非常好的深度学习平台，给大家提供免费的算力，免费的课程，各种常规比赛。里面的各种深度学习算法都有对应的模型，非常完备，工具也很齐全 给其他选手的一些建议： 
1.多参加平台上面的课程

2.多参加比赛，实践中学习

3.做项目的时候一定要多尝试各种模型，深入学习了解模型架构和原理。多尝试各种优化器，耐心调参，如果能理解各种优化器的原理和优缺点，那就更好了。

4.要学会各种数据预处理的方法，只有有了好的数据，才能训练出一个好的模型

# 参考资料
[https://www.sciencedirect.com/science/article/abs/pii/S0169260717304145](http://)
[https://agaldran.github.io/pdf/od_fovea_location.pdf](http://)